# MODELO DE PREDICCIÓN DE ENFERMEDAD CARDÍACA

## Carga y tratamiento de los datos

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Datos obtenidos de Kaggle : https://www.kaggle.com/datasets/redwankarimsony/heart-disease-data

**Column Descriptions:**  
**id** (Unique id for each patient)  
**age** (Age of the patient in years)  
**origin** (place of study)  
**sex** (Male/Female)  
**cp** chest pain type ([typical angina, atypical angina, non-anginal, asymptomatic])  
**trestbps** resting blood pressure (resting blood pressure (in mm Hg on admission to the hospital))  
**chol** (serum cholesterol in mg/dl)  
**fbs** (if fasting blood sugar > 120 mg/dl)  
**restecg** (resting electrocardiographic results)  
-- Values: [normal, stt abnormality, lv hypertrophy]  
**thalach:** maximum heart rate achieved  
**exang:** exercise-induced angina (True/ False)  
**oldpeak:** ST depression induced by exercise relative to rest  
**slope:** the slope of the peak exercise ST segment  
**ca:** number of major vessels (0-3) colored by fluoroscopy  
**thal:** [normal; fixed defect; reversible defect]  
**num:** the predicted attribute. [0=no heart disease; 1,2,3,4 = stages of heart disease ]  

In [2]:
# Leer datos CSV
data = pd.read_csv("heart_disease_uci.csv")
data.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


Comprobar numero de filas y columnas para saber si se cargó bien el fichero

In [3]:
data.shape


(920, 16)

### Análisis Descriptivo de datos


Información general del Dataframe para inferir si existen valores nulos

In [4]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        920 non-null    int64  
 1   age       920 non-null    int64  
 2   sex       920 non-null    object 
 3   dataset   920 non-null    object 
 4   cp        920 non-null    object 
 5   trestbps  861 non-null    float64
 6   chol      890 non-null    float64
 7   fbs       830 non-null    object 
 8   restecg   918 non-null    object 
 9   thalch    865 non-null    float64
 10  exang     865 non-null    object 
 11  oldpeak   858 non-null    float64
 12  slope     611 non-null    object 
 13  ca        309 non-null    float64
 14  thal      434 non-null    object 
 15  num       920 non-null    int64  
dtypes: float64(5), int64(3), object(8)
memory usage: 115.1+ KB


Se observa que existen valores nulos para las columnas :  
- trestbps  
- chol  
- fbs  
- restecg  
- thatch  
- exang  
- oldpeak  
- slope  
- ca  
- thal  


Información estadística sobre el dataset.

Esta función muestra las caracterísitcas estadísticas básicas de cada variable numérica
_number of non-missing values, mean, standard deviation, range, median, 0.25 and 0.75 quartiles._

In [5]:
data.describe()

,id,age,trestbps,chol,thalch,oldpeak,ca,num
count,920.000000,920.000000,861.000000,890.000000,865.000000,858.000000,309.000000,920.000000
mean,460.500000,53.510870,132.132404,199.130337,137.545665,0.878788,0.676375,0.995652
std,265.725422,9.424685,19.066070,110.780810,25.926276,1.091226,0.935653,1.142693
min,1.000000,28.000000,0.000000,0.000000,60.000000,-2.600000,0.000000,0.000000
25%,230.750000,47.000000,120.000000,175.000000,120.000000,0.000000,0.000000,0.000000
50%,460.500000,54.000000,130.000000,223.000000,140.000000,0.500000,0.000000,1.000000
75%,690.250000,60.000000,140.000000,268.000000,157.000000,1.500000,1.000000,2.000000
max,920.000000,77.000000,200.000000,603.000000,202.000000,6.200000,3.000000,4.000000


Conclusiones acerca de los datos:  

AGE: Se observa que existen individuos con edades entre 28 y 77 años. Se observa que la mayoría de los indiviudos están por encima de los 47 años.


trestbps: Se observa que la media está en torno a los 130 por lo que se ve que existen bastantes casos de hipertensión ( la mitad de los individuos). Ya que el q25, está en 120 que sería la presión normal en reposo.

chol : El valor normal debería estar por debajo de 200. Se observa que la media está en ese valor, el q25 está por debajo de 200 y el q50 un poco por encima por lo que existen mayor numero de individuos con valores altos de colesterol.

thalch: los valores normales de frecuencia cardíaca máxima dependen de la edad. El valor decrece con el aumento de los años. 
https://www.jacc.org/doi/10.1016/S0735-1097%2800%2901054-8#:~:text=laboratory%2Dbased%20study.-,Discussion,derived%20target%20heart%20rate%20prescriptions.

oldpeak: debe ser menor de 1mm. Se observa que la mayoría de los individuos de estudio están por debajo de ese valor.

ca : se observa que la mayoría de los pacientes no presentan bloqueos.





In [6]:
data['num'].value_counts()

num
0    411
1    265
2    109
3    107
4     28
Name: count, dtype: int64

Se observa que los datos están desbalanceados. Siendo la clase 4 la más desfavorecida. Se podría utilizar SMOTE para ajustar este desbalanceo.

#### Balanceo clases

SMOTE :  Técnica de Sobremuestreo Sintético de Minorías. Ampliación de datos para la clase minoritaria.

### Tratamiento de datos faltantes y outliers

### Análisis Exploratorio de los datos (EDA)

In [7]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

In [8]:
#Gráfico de Barras

# Preparamos una figura de m x n
mpl.rc('font', size=20)
mpl.rc('axes', titlesize=20)
figure, axes = plt.subplots(nrows=12, ncols=1) # M 3rows 2columns
figure.set_size_inches(100, 200) # Tamaño total de la figura



sns.barplot(x='age', y='num', data=data, ax=axes[0])
sns.barplot(x='sex', y='num', data=data, ax=axes[1])
sns.barplot(x='trestbps', y='num', data=data, ax=axes[2])
sns.barplot(x='chol', y='num', data=data, ax=axes[3])
sns.barplot(x='fbs', y='num', data=data, ax=axes[4])
sns.barplot(x='restecg', y='num', data=data, ax=axes[5])
sns.barplot(x='thalch', y='num', data=data, ax=axes[6])
sns.barplot(x='exang', y='num', data=data, ax=axes[7])
sns.barplot(x='oldpeak', y='num', data=data, ax=axes[8])
sns.barplot(x='slope', y='num', data=data, ax=axes[9])
sns.barplot(x='ca', y='num', data=data, ax=axes[10])
sns.barplot(x='thal', y='num', data=data, ax=axes[11])



# Titulos
axes[0].set(title='Enfermedad Cardíaca  Edad')
axes[1].set(title='Enfermedad Cardíaca Sexo')
axes[2].set(title='Presion Arterial en reposo')
axes[3].set(title='Enfermedad Cardíaca Colesterol')
axes[4].set(title='Enfermedad Cardíaca Azúcar en Ayunas>120mg/dl')
axes[5].set(title='Enfermedad Cardíaca Electro en reposo')
axes[6].set(title='Enfermedad Cardíaca Frecuencia Cardíaca Máxima')
axes[7].set(title='Angina inducida por el ejercicio')
axes[8].set(title='Depresion del segmento ST')
axes[9].set(title='Pendiente del segmento ST')
axes[10].set(title='Vasos fluoroscopia')
axes[11].set(title='Defecto')



# Rotamos 90 grados las etiquetas del eje x de la fila 1
#axes[1, 0].tick_params(axis='x', labelrotation=90)
#axes[1, 1].tick_params(axis='x', labelrotation=90)

[Text(0.5, 1.0, 'Defecto')]

In [10]:
#BOXPLOTS
# Creamos una figura de 2x2
figure, axes = plt.subplots(nrows=12, ncols=1) # 2rows 2columns
plt.tight_layout()
figure.set_size_inches(100, 300)

# Step 2 : Creamos los subgráficos
# Gráfico de bigotes de los alquileres por season, weather, holiday, and workingday


sns.boxplot(x='age', y='num', data=data, ax=axes[0])
sns.boxplot(x='sex', y='num', data=data, ax=axes[1])
sns.boxplot(x='trestbps', y='num', data=data, ax=axes[2])
sns.boxplot(x='chol', y='num', data=data, ax=axes[3])
sns.boxplot(x='fbs', y='num', data=data, ax=axes[4])
sns.boxplot(x='restecg', y='num', data=data, ax=axes[5])
sns.boxplot(x='thalch', y='num', data=data, ax=axes[6])
sns.boxplot(x='exang', y='num', data=data, ax=axes[7])
sns.boxplot(x='oldpeak', y='num', data=data, ax=axes[8])
sns.boxplot(x='slope', y='num', data=data, ax=axes[9])
sns.boxplot(x='ca', y='num', data=data, ax=axes[10])
sns.boxplot(x='thal', y='num', data=data, ax=axes[11])



# Titulos
axes[0].set(title='Enfermedad Cardíaca  Edad')
axes[1].set(title='Enfermedad Cardíaca Sexo')
axes[2].set(title='Presion Arterial en reposo')
axes[3].set(title='Enfermedad Cardíaca Colesterol')
axes[4].set(title='Enfermedad Cardíaca Azúcar en Ayunas>120mg/dl')
axes[5].set(title='Enfermedad Cardíaca Electro en reposo')
axes[6].set(title='Enfermedad Cardíaca Frecuencia Cardíaca Máxima')
axes[7].set(title='Angina inducida por el ejercicio')
axes[8].set(title='Depresion del segmento ST')
axes[9].set(title='Pendiente del segmento ST')
axes[10].set(title='Vasos fluoroscopia')
axes[11].set(title='Defecto')


# 3.2 eliminar que se sobrepongan los titulos
axes[1].tick_params('x', labelrotation=10) # rotate 10 degrees

C:\Users\mgonzalez\AppData\Local\Temp\ipykernel_30944\691911821.py:4: UserWarning: Tight layout not applied. tight_layout cannot make Axes height small enough to accommodate all Axes decorations.
  plt.tight_layout()


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,15))
plt.subplot(421)
data['age'].plot.hist(bins=10, color='blue', label='Histograma de edad', edgecolor='black')
plt.legend(loc='best')
plt.subplot(422)
data['trestbps'].plot.hist(bins=10, color='orange', label='Histograma de Presion arterial en reposo', edgecolor='black')
plt.legend(loc='best')
plt.subplot(423)
data['chol'].plot.hist(bins=10, color='pink', label='Histogram de Colesterol', edgecolor='black')
plt.legend(loc='best')
plt.subplot(424)
data['thalch'].plot.hist(bins=10, color='cyan', label='Histograma de Frecuencia cardíaca máxima', edgecolor='black')
plt.legend(loc='best')
plt.subplot(425)
data['oldpeak'].plot.hist(bins=10, color='purple', label='Histograma de Depresión del segmento ST', edgecolor='black')
plt.legend(loc='best')
plt.subplot(426)
data['ca'].plot.hist(bins=10, color='green', label='Histograma de vasos marcados', edgecolor='black')
plt.legend(loc='best')

plt.tight_layout();